Comparar KMeans "puro" com KMeans com Improved Density Canopy

Métricas erro Quadrático médio

Quando bancos de dados UCI usar?

1. Calcular MeanDis de cada elemento
2. Calcular densidade de cada elemento
3. O com maior densidade é o centro e eliminar os que não estão dentro do raio.

In [1]:
import pandas as pd
import numpy as np
from time import time

from IPython.display import display

from sklearn.cluster import KMeans

In [2]:
def euclideanDistance(vector1, vector2):
        #print(vector1)
        #print(vector2)
        return np.sqrt(np.sum(np.power(vector1-vector2, 2)))

def getDistance(row_center, row_sample):
        #print(row_center)
        row_center = np.asarray(row_center)
        #row_center = np.asarray(row_sample)
        return euclideanDistance(row_center, row_sample)


In [3]:
files = ["datasets/soybean-small.data", "datasets/wine.data"]
ks = [4,13]
kmeansTypes = ["random", "k-means++"]

In [4]:

for kmeansType in kmeansTypes:
    print ("--------- "+ kmeansType +" test ---------")
    
    for index, file in enumerate(files):
        data = pd.read_csv(file, header=None)
        print (data.shape)
        target = data.iloc[:,-1]
        data = data.iloc[:,:-1]
        #display(data.head())
    
        clustering_times = []
        start = time()
        k = ks[index]
        kmeans = KMeans(n_clusters=k, random_state=100, init=kmeansType, n_init=1, max_iter=100).fit(data)
    
        distances = []
        for i in range(k): # Qtd de clusters
            distance = 0
            for index_labels, value_labels in enumerate(kmeans.labels_): #kmeans.labels_ possui a que cluster cada elemento pertence
                if (i == value_labels):
                    #print(value_labels)
                    distance = distance + getDistance(kmeans.cluster_centers_[value_labels], data.loc[index_labels].values)
        
            distances.append(distance) #Erro quadratico medio de cada cluster
    
        distances = np.asarray(distances)
        error = np.sum(distances)
        #print(kmeans.labels_)
        end = time()
        print("Erro quadrático médio: ",error)
        clustering_times.append(end - start)
        print(clustering_times)
        #print(kmeans.cluster_centers_)

--------- random test ---------
(47, 36)
Erro quadrático médio:  100.72984563120583
[0.03965282440185547]
(178, 14)
Erro quadrático médio:  707.3637730622032
[0.05330681800842285]
--------- k-means++ test ---------
(47, 36)
Erro quadrático médio:  96.99203638192614
[0.00933074951171875]
(178, 14)
Erro quadrático médio:  674.4328770176052
[0.04040861129760742]


In [5]:

#kmeans.cluster_centers_